In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from jupyterthemes import jtplot
jtplot.style()
import statsmodels.api as sm
from patsy import dmatrices
import ipywidgets as widgets
import statsmodels.formula.api as smf

# Problem 5

Finally, we'll build some statistical models to see how well we can explain the number of aggravated burglaries using the median income of each census tract. For this, we'll be using the Generalized Linear Models module of the statsmodels library.

In [3]:
tnt = gpd.read_file('../data/tnt.geojson')

In [4]:
tnmod = (tnt[['tract', 'burgs_per_1000', 'population_density', 'gini_index', 'population', 'aland', 'median_income']]
         .drop_duplicates()
        )
tnmod = tnmod[tnmod['median_income'] >= 0]
tnmod.to_csv('../data/tnmod.csv', index = False)

a. Build a "base model" - a Poisson regression model with just an intercept term with target variable the rate of burglaries per census tract. (Offset using the [log of the] population so that we are looking at the rate of burglaries per population instead of the number of burglaries.)

In [5]:
poisreg_burg_base = (sm.GLM(endog = tnmod['burgs_per_1000'],
                       exog = sm.add_constant(tnmod[[]]),
                       family = sm.families.Poisson(),
                       offset = np.log((1/1000)*tnmod['population'])
                      )
                  .fit()
               )

In [6]:
poisreg_burg_base.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:         burgs_per_1000   No. Observations:                  146
Model:                            GLM   Df Residuals:                      145
Model Family:                 Poisson   Df Model:                            0
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -307.68
Date:                Tue, 12 Oct 2021   Deviance:                       316.55
Time:                        11:08:57   Pearson chi2:                     906.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.0925      0.068    -16.126      0.000      -1.225      -0.960
==============================================================================
"""

In [7]:
poisreg_burg_base.aic

617.3673343946223

b. Now, build a Poisson regression model with target variable the rate of burglaries and predictor variable the median income. (Don't forget to offset by the population).

In [8]:
poisreg_burg_mi = (sm.GLM(endog = tnmod['burgs_per_1000'],
                       exog = sm.add_constant(tnmod[['median_income']]),
                       family = sm.families.Poisson(),
                       offset = np.log((1/1000)*tnmod['population'])
                      )
                  .fit()
               )

In [9]:
poisreg_burg_mi.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:         burgs_per_1000   No. Observations:                  146
Model:                            GLM   Df Residuals:                      144
Model Family:                 Poisson   Df Model:                            1
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -279.01
Date:                Tue, 12 Oct 2021   Deviance:                       259.19
Time:                        11:08:57   Pearson chi2:                     682.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.3200      0.198      1.619      0.106      -0.067       0.707
median_income -2.445e-05   3.54e-06     -6.898      0.000   -3.14e-05   -1.75e-05
=================================================================================
"""

In [10]:
poisreg_burg_mi.aic

562.014148706556

c. Finally, try out a negative binomial model. To get started with a negative binomial model, you can check out this tutorial.

Start with just median_income in tract burgs, but once it works, parse the date in tn_burglaries and look for patterns.

In [11]:
#Setup the regression expression in patsy notation. 
#We are telling patsy that burgs_per_1000 is our dependent variable and it depends on the regression variables: median_income
tnmod2 = tnmod.copy()
expr = "burgs_per_1000 ~ median_income"

#Set up the X and y matrices for the training and testing data sets
y_train, X_train = dmatrices(expr, tnmod2, return_type='dataframe')

#Using the statsmodels GLM class, train the Poisson regression model on the training data set
poisson_training_results = (sm.GLM(y_train, 
                                  X_train, 
                                  family=sm.families.Poisson(),
                                  offset = np.log((1/1000)*tnmod2['population'])
                                  )
                                  .fit()
                            )

#print out the training summary
print(poisson_training_results.summary())

#print out the fitted rate vector
#print(poisson_training_results.mu)

#Add the λ vector as a new column called 'BB_LAMBDA' to the Data Frame of the training data set
tnmod2['TB_LAMBDA'] = poisson_training_results.mu

#add a derived column called 'AUX_OLS_DEP' to the pandas DataFrame. This new column will store the values of the dependent variable of the OLS regression
tnmod2['AUX_OLS_DEP'] = tnmod2.apply(lambda x: ((x['burgs_per_1000'] - x['TB_LAMBDA'])**2 - x['TB_LAMBDA']) / x['TB_LAMBDA'], axis=1)

#use patsy to form the model specification for the OLSR
ols_expr = """AUX_OLS_DEP ~ TB_LAMBDA - 1"""

#Configure and fit the OLSR model
aux_olsr_results = smf.ols(ols_expr, tnmod2).fit()

#Print the regression params
print(aux_olsr_results.params)

#train the NB2 model on the training data set
nb2_training_results = sm.GLM(y_train, X_train,family=sm.families.NegativeBinomial(alpha=aux_olsr_results.params[0])).fit()

#print the training summary
print(nb2_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:         burgs_per_1000   No. Observations:                  146
Model:                            GLM   Df Residuals:                      144
Model Family:                 Poisson   Df Model:                            1
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -279.01
Date:                Tue, 12 Oct 2021   Deviance:                       259.19
Time:                        11:08:57   Pearson chi2:                     682.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.3200      0.198      1.619

d. How do your models compare? Hint: the fit models have an AIC attribute.

In [12]:
#Check AIC
print("poisson: "+str(poisson_training_results.aic)) 
print("negative binomial: "+str(nb2_training_results.aic))

poisson: 562.014148706556
negative binomial: 482.17972219341664


# Function and Widget

In [15]:
def nb2(data, target, offset_col, exp_vars, print_summary = False):
    """Create NB2 model"""
    #Copy data to df
    df = data.copy()
    #Setup the regression expression in patsy notation. 
    #We are telling patsy that burgs_per_1000 is our dependent variable and it depends on the regression variables: median_income
    expr = f"{target} ~ {' + '.join(exp_vars)}"

    #Set up the X and y matrices for the training and testing data sets
    y_train, X_train = dmatrices(expr, df, return_type='dataframe')

    #Using the statsmodels GLM class, train the Poisson regression model on the training data set
    poisson_training_results = (sm.GLM(y_train, 
                                      X_train, 
                                      family=sm.families.Poisson(),
                                      offset = np.log((1/1000)*df[offset_col])
                                      )
                                      .fit()
                                )

    #print out the fitted rate vector
    #print(poisson_training_results.mu)

    #Add the λ vector as a new column called 'BB_LAMBDA' to the Data Frame of the training data set
    df['TB_LAMBDA'] = poisson_training_results.mu

    #add a derived column called 'AUX_OLS_DEP' to the pandas DataFrame. This new column will store the values of the dependent variable of the OLS regression
    df['AUX_OLS_DEP'] = df.apply(lambda x: ((x[target] - x['TB_LAMBDA'])**2 - x['TB_LAMBDA']) / x['TB_LAMBDA'], axis=1)

    #use patsy to form the model specification for the OLSR
    ols_expr = """AUX_OLS_DEP ~ TB_LAMBDA - 1"""

    #Configure and fit the OLSR model
    aux_olsr_results = smf.ols(ols_expr, df).fit()

    #Print the regression params
    #print(aux_olsr_results.params, "\n\n\n")

    #train the NB2 model on the training data set
    nb2_training_results = (sm.GLM(y_train, 
                                  X_train,
                                  family=sm.families.NegativeBinomial(alpha=aux_olsr_results.params[0]))
                           .fit()
                           )
    
    #print the training summary
    if print_summary == True:
        print("POISSON SUMMARY")
        print(poisson_training_results.summary(), "\n\n\n")
    
        print("NB2 SUMMARY")
        print(nb2_training_results.summary(), '\n\n\n')
        print(' Poisson AIC: '+str(poisson_training_results.aic), 
              '\n',
              'NB2 AIC: '+str(nb2_training_results.aic)
             )
    else:
        pass
    
    return (poisson_training_results, nb2_training_results)

In [19]:
ALL = 'ALL'
selist = tnmod.columns[2:].tolist()
selist.append(ALL)
select_list = selist[::-1]

w = widgets.SelectMultiple(
    options=select_list,
    value=['ALL'],
    rows=6,
    description='Exp. Vars',
    disabled=False
)

def widfunc(x):
    if x != ('ALL',):
        exp_vars = x
    else:
        exp_vars = tnmod.columns[2:].tolist()
        
    return nb2(data = tnmod,
            target = 'burgs_per_1000',
            offset_col = 'population',
            exp_vars = exp_vars,
            print_summary = True)

widgets.interact(widfunc, 
                 x = w)

interactive(children=(SelectMultiple(description='Exp. Vars', index=(0,), options=('ALL', 'median_income', 'al…

<function __main__.widfunc(x)>